In [17]:
# !pip install cellrank
# !pip install scanpy
# !pip install scipy
# !pip install anndata


In [7]:
%%bash

source downloadData.sh

# !mkdir -p ./data

# SUESS dataset
download_dataset "https://www.ncbi.nlm.nih.gov/geo/download/?acc=GSE107185&format=file" "./data" "GSE107185_RAW.tar"
tar -xvf ./data/GSE107185_RAW.tar -C ./data


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed


100  164M  100  164M    0     0  1503k      0  0:01:52  0:01:52 --:--:-- 2022k


GSM3308844_up-tf-stem-sample2.counts.tsv.gz
GSM3308844_up-tf-stem-sample2_pheno_dict.csv.gz
GSM3308845_up-tf-stem-sample3.counts.tsv.gz
GSM3308845_up-tf-stem-sample3_pheno_dict.csv.gz
GSM3308846_up-tf-endo-sample1.counts.tsv.gz
GSM3308846_up-tf-endo-sample1_pheno_dict.csv.gz
GSM3308847_up-tf-endo-sample2.counts.tsv.gz
GSM3308847_up-tf-endo-sample2_pheno_dict.csv.gz
GSM3308848_up-tf-multi-sample1.counts.tsv.gz
GSM3308848_up-tf-multi-sample1_pheno_dict.csv.gz
GSM3308849_up-tf-multi-sample2.counts.tsv.gz
GSM3308849_up-tf-multi-sample2_pheno_dict.csv.gz
GSM3308850_up-tf-klf-myc_pheno_dict.csv.gz
GSM3308850_up-tf-klf.counts.tsv.gz
GSM3308850_up-tf-myc.counts.tsv.gz
GSM3308851_up-tf-neuron-hygro-sample1.counts.tsv.gz
GSM3308851_up-tf-neuron-hygro-sample1_pheno_dict.csv.gz
GSM3308852_up-tf-neuron-hygro-sample2.counts.tsv.gz
GSM3308852_up-tf-neuron-hygro-sample2_pheno_dict.csv.gz
GSM3308853_up-tf-neuron-nohygro-sample1.counts.tsv.gz
GSM3308853_up-tf-neuron-nohygro-sample1_pheno_dict.csv.gz
GSM

In [3]:
import cellrank as  cr
import scanpy as sc
from scipy import sparse
import pandas as pd
import numpy as np
from anndata import AnnData

 Transcript abundance (TPM) measurements for induced pluripotent stem cells (iPSCs), COV434 ovarian cancer cells, and iPSC-derived granulosa-like cells. 2. Differential gene expression analysis to identify effects of transcription factor overexpression during iPSC differentiation.

In [8]:

df = pd.read_csv('data/GSE107185_RAW.csv', sep=',')

print(df.head())
print(df2.head())


# the nine TFs are:
    # WT1
    # GATA4
    # NR5A1
    # RUNX1
    # FOXL2
    # TCF21
    # COV434
    # FOXL2_plus
    # NOTFS


# # Create anndata object
# adata = AnnData(X=df.values, obs=pd.DataFrame(index=df.index), var=pd.DataFrame(index=df.columns))

# print(adata.obs.head())
# print(adata.var.head())

# this data is just the abundance of the log2fold change of the TFs. Does not have transcription information? 


FileNotFoundError: [Errno 2] No such file or directory: 'data/GSE107185_RAW.csv'